<a href="https://colab.research.google.com/github/MariOrki/tesis/blob/main/bbdd_NLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pandas
! pip install geopandas
! pip install plotly.express
! pip install matplotlib
! pip install geojson
! pip install ipykernel
#! pip install "jupyterlab>=3" "ipywidgets>=7.6"
! pip install plotly #quizás no es necesario con plotly express?
! pip install ast
! pip install nbformat 
! pip install --upgrade nbformat
! pip install geoplot
! pip install statsmodels
! pip3 install geojson 

In [2]:
! pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 74.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import plotly.graph_objects as go
import geopandas as gpd 
import matplotlib.pyplot as plt
import plotly.express as px #annotations
import plotly.graph_objs as pg
import datetime
import numpy as np
import scipy.interpolate
import statsmodels
import requests
import os
from bs4 import BeautifulSoup

In [ ]:
import geojson
from geojson import GeometryCollection, Point, LineString, Feature, FeatureCollection
import ast

In [8]:
ls

anscombe.json*                mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md*


In [ ]:


folder_path = 'Total'  # Ruta de la carpeta principal
output_dataframes = {}  # Diccionario para almacenar los DataFrames resultantes

# Obtener la lista de archivos y subcarpetas en la carpeta principal
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.csv') and file.startswith('CR2_NLU_') and 'historical_1950-2020_communes' in file:
            # Obtener el nombre de la subcarpeta y el nombre del archivo
            subfolder = os.path.basename(root)
            filename = os.path.splitext(file)[0]
            name_parts = filename.split('_')

            # Construir el nombre deseado para el DataFrame
            dataframe_name = subfolder + '_' + name_parts[-2]

            # Leer el archivo CSV en un DataFrame
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path)

            # Asignar el DataFrame al diccionario usando el nombre deseado
            output_dataframes[dataframe_name] = df

# Acceder a los DataFrames resultantes por su nombre
total_communes = output_dataframes['Total_communes']
drinkingwater_communes = output_dataframes['DrinkingWater_communes']
energy_communes = output_dataframes['Energy_communes']
industry_communes = output_dataframes['Industry_communes']

# Realizar operaciones adicionales con los DataFrames según sea necesario

In [ ]:
ch = gpd.read_file(r'/mnt/home2/mari/Tesis/NLU/GeoJSON/nation_ContinentalChile_epsg4326.geojson')
reg = gpd.read_file(r'/mnt/home2/mari/Tesis/NLU/GeoJSON/regions_ContinentalChile_epsg4326.geojson')
com = gpd.read_file(r'/mnt/home2/mari/Tesis/NLU/GeoJSON/communes_ContinentalChile_epsg4326.geojson')

In [ ]:
#file containing codes and names
cod = pd.read_csv(r'/mnt/home2/mari/Tesis/NLU/LUT_cod_com_Chile.csv')

#dictionary code region : name region
cod_reg_nom_reg = cod[['cod_reg', 'nom_reg']].drop_duplicates().set_index('cod_reg')['nom_reg'].astype(str).to_dict()
cod_reg_nom_reg = {str(k): v for k, v in cod_reg_nom_reg.items()} #same type of data as csv column names NLU_XX

#dictionary code commune : name commune
cod_com_nom_com = cod[['cod_com', 'nom_com']].drop_duplicates().set_index('cod_com')['nom_com'].astype(str).to_dict()
cod_com_nom_com = {str(k): v for k, v in cod_com_nom_com.items()} 

In [ ]:
years_with_365_days = []
years_with_366_days = []

for year in range(1950, 2021):
    date = datetime.date(year, 12, 31)  # Crear una fecha para el último día del año
    
    if date.timetuple().tm_yday == 365:
        years_with_365_days.append(year)

    if date.timetuple().tm_yday == 366:
        years_with_366_days.append(year)

y_365=[str(item) for item in years_with_365_days]
y_366=[str(item) for item in years_with_366_days]

In [ ]:
cd sample_data

/content/sample_data


In [ ]:
NLU_nation = pd.read_csv ('CR2WU_NLU_Total_historical_1950-2020_nation.csv')
NLU_regions = pd.read_csv ('CR2WU_NLU_Total_historical_1950-2020_regions.csv')

In [ ]:
#Database cleaning
NLU_nation['Date'] = pd.to_datetime(NLU_nation["Date"])
NLU_nation['Date'] = NLU_nation['Date'].dt.strftime('%Y')
NLU_nation = NLU_nation.rename(columns={"Date": "Years","1":"NLU"})
NLU_nation['Years'] = NLU_nation['Years'].astype(int)

NLU_regions['Date'] = pd.to_datetime(NLU_regions["Date"])
NLU_regions['Date'] = NLU_regions['Date'].dt.strftime('%Y')
NLU_regions = NLU_regions.rename(columns={"Date": "Years"})
NLU_regions['Years'] = NLU_regions['Years'].astype(int)

#Adding Region's names
NLU_regions_plot = NLU_regions.rename(columns=cod_reg_nom_reg)

#NLU national 2 decimal
NLU_nation_ = NLU_nation
NLU_nation_['NLU'] = NLU_nation_['NLU'].round(decimals=2)

In [ ]:
NLU_nation.head()

,Years,NLU
0,1950,371.983515
1,1951,372.250363
2,1952,372.545652
3,1953,373.614361
4,1954,374.197526


In [ ]:
fig1 = px.line(NLU_nation, x='Years', y='NLU', title="Uso de agua no relacionado con la tierra en Chile", labels={'Years':'Años', 'NLU' : 'Uso de agua [m³/s]'})

fig1.show()

In [ ]:
fig2 = px.line(NLU_nation, x='Years', y='NLU', title="Non Land Water Use in Chile", labels={'Years':'Years', 'NLU' : 'Water use [m³/s]'})

fig2.show()

In [ ]:
#Useful lists
year_list = NLU_regions['Years'].tolist()
reg_list = [i for i in cod_reg_nom_reg.values()] #not ordered by lat
com_list = [i for i in cod_com_nom_com.values()] #not ordered by lat

In [ ]:
total_use = NLU_nation['NLU'].sum()
#total_use_km3 = NLU_nation['NLU_year'].sum()

#Min and max
year_max = NLU_nation.loc[NLU_nation['NLU'].idxmax(), 'Years']
year_min = NLU_nation.loc[NLU_nation['NLU'].idxmin(), 'Years']
use_max = NLU_nation['NLU'].max()
use_min = NLU_nation['NLU'].min()

#Mean use in the last 10 years
last_years = (NLU_nation['Years'] >= 2010) & (NLU_nation['Years'] <= 2020)
NLU_last_years = NLU_nation[last_years]
mean_use = NLU_last_years['NLU'].mean()
#mean_use_km3 = NLU_last_years['NLU_year'].mean()

print("Total water use:", f"{total_use:.1f}""[m³/s]") #quizás esto no tiene mucho sentido
#print("Total water use:", f"{total_use_km3:.1f}""[km³]")
print("Highest water use year:", f"{use_max:.1f}" "[m³/s]", "in", year_max)
print("Lowest water use year:", f"{use_min:.1f}" "[m³/s]", "in",year_min)
print("Mean water during the last 10 years:", f"{mean_use:.1f}""[m³/s]")
#print("Mean water during the last 10 years per year:", f"{mean_use_km3:.1f}""[km³/y]")

In [ ]:
#Según el Informe Mundial de las Naciones Unidas sobre el Desarrollo de los Recursos Hídricos, 
#el consumo total de agua dulce a nivel mundial se estima en alrededor de 4.600 kilómetros cúbicos (km³) por año.
#y según Naciones Unidas, existen 195 países

#4600/195 = 23.6 => cada país usa 23.6[km3] de agua dulce al año
#92.2/23.6 =3.9 => pero Chile, usa casi 4 veces ese volumen (92.2[km3] usados en promedio entre 2010 y 2020) 

In [ ]:
#Graph of yearly national (Non Land) Water Use [m3/s]

#English
fig = px.scatter(NLU_nation, x="Years", y="NLU", trendline='ols',
              title="Non Land Water Use in Chile", 
              labels={'Years':'Years', 'NLU':'Water use [m³/s]'}) 
fig.show()

#Spanish
fig2 = px.line(NLU_nation, x="Years", y="NLU", 
               title="Uso de agua no relacionado con la tierra", 
               labels={'Years':'Años', 'NLU':'Uso de agua [m³/s]'})
fig2.show()

National Water Use and Inhabitants

In [ ]:
# hacerlo para un path que sí tenga acceso
#pop = pd.read_csv(r'/mnt/home2/mari/Tesis/INE_2002-2035.csv', index_col=0, sep=';', encoding='latin-1')

In [ ]:
#Refining Dataframe Population

#Eliminate columns
col_drop = ['Sexo (1=Hombre 2=Mujer)', 'Grupo edad']
pop = pop.drop(col_drop,axis=1)

#Changing column names
pop_grouped = pop.groupby(['Nombre Comuna', 'Area (1=Urbano 2=Rural)'])[pop.columns[6:39]].sum().reset_index()
pop = pop.rename(columns=lambda x: x.replace('Poblacion ', '') if x.startswith('Poblacion ') else x) #year columns
pop = pop.rename(columns=lambda x: x.replace('Nombre ', 'nom_') if x.startswith('Nombre ') else x) 
pop = pop.rename(columns={'Provincia':'cod_provincia'})
pop = pop.rename(columns=lambda x: x.replace('Region', 'region') if 'Region' in x else x)
pop = pop.rename(columns=lambda x: x.replace('Provincia', 'provincia') if 'Provincia' in x else x)
pop = pop.rename(columns=lambda x: x.replace('Comuna', 'comuna') if 'Comuna' in x else x)
pop = pop.rename(columns={'Area (1=Urbano 2=Rural)':'urban1_rural2'})
pop.index.name = 'region'

In [ ]:
#Tengo que hacerlo para pop grouped, lo borré sin querer
#Population and NL Water Use Dataframe
pop_tot = pop.iloc[:, 6:25].sum().astype(int) #total population
#pop_tot = pop.iloc[:, 6:25].shift().sum().astype(int)
pop_years = pd.DataFrame({'Years': pop_tot.index, 'Population': pop_tot.values})
pop_years.head()

In [ ]:
NLU_pop = NLU_nation_[NLU_nation_['Years'] >= 2002].reset_index(drop=True) #using NLU_nation_ 2 decimals NLU
#NLU_pop = NLU_nation_[NLU_nation_['Years'] >= 2002].reset_index(drop=True) #using NLU_nation_ 2 decimals NLU
#NLU_pop['Population']=pop_years_g['Population'] #usar este cuando corrija lo de arriba (crear pop years grouped)
NLU_pop['Population']=pop_years['Population']
NLU_pop.head()

In [ ]:
fig = px.scatter(NLU_pop, x="Population", y="NLU", trendline = 'ols',
              title="Population and Water Use in Chile", 
              labels={'Population':'Population', 'NLU':'Water use [km³]'}) 



fig2 = go.Figure(data=go.Scatter(x=NLU_pop["Population"], y=NLU_pop["NLU"]))
fig2.update_traces(line=dict(color='red'))

fig.show()
fig2.show()

In [ ]:

NLU_pop['Log_Pop']=np.log(NLU_pop['Population'])
NLU_pop['Log_NLU']=np.log(NLU_pop['NLU'])
fig = px.line(NLU_pop, x="Log_Pop", y="Log_NLU", 
              title="Population and Water Use in Chile", 
              labels={'Log_Pop':'Log Population', 'NLU':'Log Water use'}) 

fig.show()
fig.write_image(r"/mnt/home2/mari/Tesis/logpopul_logwater.png")

Regional Water Use

In [ ]:
#Graph of yearly regional (Non Land) Water Use [m3/s]

x_column = NLU_regions_plot.columns[0]
NLU_regions_melted = NLU_regions_plot.melt(id_vars=x_column)
fig = px.bar(NLU_regions_melted, x=x_column, y='value', color='variable',
             barmode='stack', title = "Non Land Water Use in Chile",labels={'Years':'Years', 'value':'Water use [m³/s]'}) 
fig.update_layout(legend_title_text="Regions")

fig.show()

fig2 = px.bar(NLU_regions_melted, x=x_column, y='value', color='variable',
             barmode='stack', title="Uso de agua no relacionado con la tierra en CHile", labels={'Years':'Años', 'value':'Uso de agua [m³/s]'}) 
fig2.update_layout(legend_title_text="Regiones")

fig2.show()

Regional Water Use Analysis

In [ ]:
#Last 10 years WU
NLU_regions_plot2 = NLU_regions_plot.loc[NLU_regions_plot['Years'] > 2010]

In [ ]:
#Regional latitude order
ord_col = ['Years','REGIÓN DE ARICA Y PARINACOTA','REGIÓN DE TARAPACÁ', 'REGIÓN DE ANTOFAGASTA',
       'REGIÓN DE ATACAMA', 'REGIÓN DE COQUIMBO', 'REGIÓN DE VALPARAÍSO','REGIÓN METROPOLITANA DE SANTIAGO',
       "REGIÓN DEL LIBERTADOR GENERAL BERNARDO O'HIGGINS", 'REGIÓN DEL MAULE','REGIÓN DE ÑUBLE',
       'REGIÓN DEL BIOBÍO', 'REGIÓN DE LA ARAUCANÍA', 'REGIÓN DE LOS RÍOS', 'REGIÓN DE LOS LAGOS',
       'REGIÓN DE AYSÉN DEL GENERAL CARLOS IBÁÑEZ DEL CAMPO',
       'REGIÓN DE MAGALLANES Y DE LA ANTÁRTICA CHILENA']

NLU_regions_plot2 = NLU_regions_plot2.reindex(columns=ord_col)

mean_values = NLU_regions_plot2.iloc[:, 1:].shift().mean()
#mean_row = pd.DataFrame([mean_values], columns=NLU_regions_plot2.columns[1:])

fig = px.bar(mean_values, x=mean_values.index, y=mean_values.values, 
             title='Yearly mean water use during the last 10 years by region ordered from north to south',
            labels={'index':'','y':'Water use [m³/s]'})
fig.update_xaxes(tickangle=60, tickfont=dict(size=9))
fig.update_layout(width = 1000, height = 800)
fig.show()

Type of Use Analysis

In [ ]:
DW_nation = pd.read_csv(r'/mnt/home2/mari/Tesis/NLU/csv/Total/DrinkingWater/CR2WU_NLU_DrinkingWater_historical_1950-2020_nation.csv')
DW_regions = pd.read_csv(r'/mnt/home2/mari/Tesis/NLU/csv/Total/DrinkingWater/CR2WU_NLU_DrinkingWater_historical_1950-2020_regions.csv')

In [ ]:
#Total use = DW+Energy+....
Total_nation = pd.read_csv(r'/mnt/home2/mari/Tesis/NLU/csv/Total/CR2WU_NLU_Total_historical_1950-2020_nation.csv')
Energy_nation = pd.read_csv(r'/mnt/home2/mari/Tesis/NLU/csv/Total/Energy/CR2WU_NLU_Energy_historical_1950-2020_nation.csv')
Industry_nation = pd.read_csv(r'/mnt/home2/mari/Tesis/NLU/csv/Total/Industry/CR2WU_NLU_Industry_historical_1950-2020_nation.csv')
Livestock_nation = pd.read_csv(r'/mnt/home2/mari/Tesis/NLU/csv/Total/Livestock/CR2WU_NLU_Livestock_historical_1950-2020_nation.csv')
Mining_nation = pd.read_csv(r'/mnt/home2/mari/Tesis/NLU/csv/Total/Mining/CR2WU_NLU_Mining_historical_1950-2020_nation.csv')
WaterBodies_nation = pd.read_csv(r'/mnt/home2/mari/Tesis/NLU/csv/Total/WaterBodies/CR2WU_NLU_WaterBodies_historical_1950-2020_nation.csv')

In [ ]:
#suma sin purificar 
E_tot = Energy_nation['1'].sum()
I_tot = Industry_nation['1'].sum()
L_tot = Livestock_nation['1'].sum()
M_tot = Mining_nation['1'].sum()
W_tot = WaterBodies_nation['1'].sum()
D_tot = DW_nation['1'].sum()
Total_tot = Total_nation['1'].sum()
sum_tot = D_tot+E_tot+I_tot+L_tot+M_tot+W_tot
Total_tot-sum_tot #-1.4551915228366852e-11 diría que es error de truncamiento

In [ ]:
#Database cleaning
DW_nation['Date'] = pd.to_datetime(DW_nation["Date"])
DW_nation['Date'] = DW_nation['Date'].dt.strftime('%Y')
DW_nation = DW_nation.rename(columns={"Date": "Years","1":"NLU"})
DW_nation['Years'] = DW_nation['Years'].astype(int)

DW_regions['Date'] = pd.to_datetime(DW_regions["Date"])
DW_regions['Date'] = DW_regions['Date'].dt.strftime('%Y')
DW_regions = DW_regions.rename(columns={"Date": "Years"})
DW_regions['Years'] = DW_regions['Years'].astype(int)

#Adding Region's names
DW_regions_plot = DW_regions.rename(columns=cod_reg_nom_reg)

#NLU national 2 decimal
DW_nation_ = DW_nation
DW_nation_['NLU'] = DW_nation_['NLU'].round(decimals=2)

In [ ]:
DW_pop = DW_nation_[DW_nation_['Years'] >= 2002].reset_index(drop=True) #using NLU_nation_ 2 decimals NLU
#NLU_pop['Population']=pop_years_g['Population'] #usar este cuando corrija lo de arriba (crear pop years grouped)
DW_pop['Population']=pop_years['Population']
DW_pop.head()

In [ ]:
fig = px.scatter(DW_pop, x="Population", y="NLU", 
              title="Population and Drinking Water Use in Chile", 
              labels={'Population':'Population', 'NLU':'Drinking water use [m³/s]'}) 

fig.show()
#fig.write_image(r"/mnt/home2/mari/Tesis/pop_dw.png")

In [ ]:
DW_pop['Log_Pop']=np.log(DW_pop['Population'])
DW_pop['Log_NLU_DW']=np.log(DW_pop['NLU'])
fig = px.scatter(DW_pop, x="Log_Pop", y="Log_NLU_DW", 
              title="Population and Drinking Water Use in Chile", 
              labels={'Log_Pop':'Log Population', 'NLU':'Log Drinking Water use'}) 

fig.show()
#fig.write_image(r"/mnt/home2/mari/Tesis/logpopul_logdw.png")

In [ ]:
fig = px.scatter(DW_pop, x="Years", y="NLU/Pop", trendline="ols",
              title="Population and Drinking Water Use in Chile", 
              labels={'Years':'Years', 'NLU/Pop':'Drinking Water use per cápita [m³/s pp]'})
fig.update_layout(yaxis_tickformat = '.1e')


fig.show()

In [ ]:
DW_pop['NLU/Pop']=DW_pop['NLU']/DW_pop['Population']
DW_pop['Log_NLU/Pop' ]=np.log(DW_pop['NLU/Pop'])
fig = px.scatter(DW_pop, x="Log_Pop", y="Log_NLU/pop", 
              title="Population and Drinking Water Use in Chile", 
              labels={'Log_Pop':'Log Population', 'Log_NLU/pop':'Log Drinking Water use per cápita'})

fig.show()